In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.optim as optim
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode


In [ ]:
#학습에서 사용되는 수치들
#Batch size 
batch_size=128 #모두 로드를 할 수 없으니 한번에 배치사이즈만큼 가져옴
#Epoch  
num_epochs=3 #모델이 훈련에 얼마만큼 데이터를 받는지
#learning rate
learning_rate=0.001 #학습률이 높을수록 빠르게 학습할 수 있지만 학습이 다른 쪽으로 갈 수도 있음(신중하지 못함)

In [ ]:
data_transforms = {
    'train': transforms.Compose([ #train 전처리
        transforms.RandomResizedCrop(224), #224사이즈로 크롭
        transforms.RandomHorizontalFlip(), 
        transforms.ToTensor(), #들어온 이미지에 대해 텐서로 변경
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화
    ]),
    'val': transforms.Compose([ #val - test set 전처리
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_set=torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transforms['train']) #cifar10 로드
test_set =torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transforms['val'])

dataloaders=dict()
dataloaders['train']= torch.utils.data.DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True) #데이터로더로 가져옴 train과
dataloaders['val']= torch.utils.data.DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False) #test, test에서는 데이터 순서를 셔플 해 줄 필요 없음

dataset_sizes = {x: len(dataloaders[x].dataset) for x in ['train', 'val']}

print("train 개수",dataset_sizes['train'])
print("test 개수",dataset_sizes['val'])

class_names = train_set.classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("class_names:",class_names) #cifar10의 클래스명 airplain.. 이 들어감
print(device)

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor.""" #텐서 형태로 된 이미지를 볼 수 있는 함수
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train'])) #몇개 가져와서 imshow로 확인해보기
#batch가 너무 크면 다 안보이니 3개만 가져오기
inputs_=inputs[:3]
classes_=classes[:3]

# Make a grid from batch
out = torchvision.utils.make_grid(inputs_)

imshow(out, title=[class_names[x] for x in classes_])

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25): #모델을 실제로 train 하는 코드
    since = time.time() #모델이 학습할 때 얼마나 걸리는지 계산함

    best_model_wts = copy.deepcopy(model.state_dict()) #모델의 가중치에 대한 정보들이 담긴state dict를 깊은 복사를 해옴, 
    #모델이 제일 좋은 성능을 보일때 가중치를 저장
    best_acc = 0.0 #best accuracy

    train_loss_list=[]
    val_acc_list=[]

    for epoch in range(num_epochs): # num epoochs만큼 반복학습
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

#파이토치에서는 epoch마다 train과 성능 evaluation
        # Each epoch has a training and validation phase 
        for phase in ['train', 'val']: # train을 하고나서 train 된 모델에 대해 test하기 위해서 val이 들어감
            if phase == 'train':
                model.train()  # Set model to training mode 학습으로 모델 모드 변경
            else:
                model.eval()   # Set model to evaluate mode 평가로 모델 모드 변경

            running_loss = 0.0
            running_corrects = 0
            iteration_count=0
            # Iterate over data.
            for inputs, labels in dataloaders[phase]: #input이미지, labels클래스 #phase가 val 로 test를 함
                iteration_count+=len(inputs)
                print('Iteration {}/{}'.format(iteration_count,dataset_sizes[phase]))
                inputs = inputs.to(device)
                labels = labels.to(device) #input과 labels를 gpu상에 올려서 연산이 가능하도록 만들어줌

                # zero the parameter gradients
                optimizer.zero_grad() #그래디언트 파라미터를 0으로 만들어줌

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) #예측 확률이(얼마나 맞췄는지) 가장 높은게 저장됨
                    loss = criterion(outputs, labels) #로스를 구함

                    # backward + optimize only if in training phase
                    if phase == 'train': #훈련상태면 (test는 loss가 사용 안되니까 loss갱신 필요 업음)
                        loss.backward() #구한 로스를 각 노드들로 전달
                        optimizer.step() #optimizer로 가중치를 갱신 

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step() #스케쥴러에 대해서도 step으로 갱신 ex) 7epoch마다 0.1씩 곱해줘서 학습률을 바꿔줬는데 이 스케줄러로 갱신됨

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase] #에폭에 대한 loss와 accuracy

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            if phase=="train":
              train_loss_list.append(epoch_loss) #얼마나 차이가 나타나는지
            elif phase=="val":
              val_acc_list.append(epoch_acc)


            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc: #기존보다 test의 정확도가 더 좋았다면
                best_acc = epoch_acc #best를 현재로 갱신해줌
                best_model_wts = copy.deepcopy(model.state_dict()) #현재 모델의 가중치 정보도 넣어줌

        print()

    time_elapsed = time.time() - since #학습에 소요된 시간
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts) #가장 좋은 성능을 보였던 모델의 가중치를 로드해줌
    return model,train_loss_list,val_acc_list

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
model_ft = models.vgg16(pretrained=True) #앞서 model에서작성한 vgg16을 호출, 사전학습이 된 가중치를 vgg16에 로드해서 가져옴
num_ftrs = model_ft.classifier[6].in_features #6번째 in_features인 4096을 가져옴 , imagenet이 아닌 cifar10으로 할거임
model_ft.classifier[6] = nn.Linear(num_ftrs, len(class_names)) #클래스 개수에 맞춰 output 디맨션 맞춰줌
model_ft = model_ft.to(device) #device는 주로gpu
print(model_ft)
#여기까지 모델의 최종 출력층을 바꿔줬다면


criterion = nn.CrossEntropyLoss() #LOSS FUNC을 어떤걸 사용할 지 정의

# Observe that all parameters are being optimized
#SGD
optimizer_ft = optim.SGD(model_ft.parameters(), lr=learning_rate, momentum=0.9) #옵티마이저로 sgd 사용
#Adam
# optimizer_ft = optim.Adam(model_ft.parameters(), lr=learning_rate)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1) #lr ~ stepLR을 사용해 학습률을 줄임. 감마값 0.1씩을 곱해주면서 바꿔줌

In [ ]:
model_ft,train_loss_list,val_acc_list = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=num_epochs) #train_model로 몇에폭을 학습할지 등등을 정해 학습함

In [ ]:
#plot train loss 
x=[i for i in range(0,num_epochs)]
plt.title("Train Loss")
plt.xticks(x)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(x,train_loss_list) 
plt.show()

#plot test acc
x=[i for i in range(0,num_epochs)]
plt.title("Test Accuracy")
plt.xticks(x)
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.plot(x,val_acc_list)
plt.show()

In [ ]:
visualize_model(model_ft)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!jupyter nbconvert --to html "/content/drive/MyDrive/Colab Notebooks/VGGNet_train_pytorch.ipynb"